In [1]:
from sklearn.metrics import classification_report
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.models import Model as KerasModel
from tensorflow.python.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.python.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from pprint import pprint
random.seed(32)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3" # Choose GPU NUMBERS [0, 1, 2, 3]
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

In [2]:
def generate_generator_multiple(generator,dir1, dir2,dir3,dir4,dir5, batch_size, img_height,img_width):
    genX1 = generator.flow_from_directory(dir1,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)   
    genX2 = generator.flow_from_directory(dir2,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
    genX3 = generator.flow_from_directory(dir3,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
    genX4 = generator.flow_from_directory(dir4,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
    genX5 = generator.flow_from_directory(dir5,
                                          target_size = (img_height,img_width),
                                          class_mode = 'categorical',
                                          batch_size = batch_size,
                                          shuffle=True, 
                                          seed=32)
#     for i in range(0,len(genX1.filepaths)+len(genX2.filepaths)):
    i=0
    while(True):
        if i<len(genX1.filepaths):
            X1i = genX1.next()
            yield X1i[0], X1i[1]
            
        if i<len(genX2.filepaths):
            X2i = genX2.next()
            yield X2i[0], X2i[1] 
            
        if i<len(genX3.filepaths):
            X3i = genX3.next()
            yield X3i[0], X3i[1]
            
        if i<len(genX4.filepaths):
            X4i = genX4.next()
            yield X4i[0], X4i[1] 
            
        if i<len(genX5.filepaths):
            X5i = genX5.next()
            yield X5i[0], X5i[1]
        i+=1

In [3]:
batch_size=200
img_height=128
img_width=128
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator=generate_generator_multiple(generator=train_datagen,
                                           dir1='datasets/DeepFake/train',
                                           dir2='datasets/FaceSwap/train',
                                           dir3='datasets/Face2Face/train',
                                           dir4='datasets/NeuralTextures/train',
                                           dir5='datasets/DeepFakeDetection/train',
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_width)    
validation_generator=generate_generator_multiple(validation_datagen,
                                          dir1='datasets/DeepFake/val',
                                          dir2='datasets/FaceSwap/val',
                                          dir3='datasets/Face2Face/val',
                                          dir4='datasets/NeuralTextures/val',
                                          dir5='datasets/DeepFakeDetection/val',
                                          batch_size=batch_size,
                                          img_height=img_height,
                                          img_width=img_width) 

In [4]:
x=train_generator.__next__()
print(x[1].shape)
pprint(x[1])
plt.imshow(x[0][1])
plt.title(x[1][1])

In [5]:
x=validation_generator.__next__()
print(x[1].shape)
pprint(x[1])
plt.imshow(x[0][1])
plt.title(x[1][1])

In [6]:
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model_checkpoint_callback = ModelCheckpoint(
    filepath="XCE_ALL_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("XCE_ALL_train.csv", append=True, separator=',')
print_summary(model, line_length=150, positions=None, print_fn=None)

In [7]:
total_train=520000/8 #65,000
total_val=84000/8   #10,500
model.fit_generator(train_generator,
                    verbose=1,
                    steps_per_epoch=int(total_train/batch_size),
                    epochs=50,
                    validation_data=validation_generator,
                    validation_steps = int(total_val/batch_size),
                    callbacks=[model_checkpoint_callback,csv_logger])
model.save_weights("XCE_ALL_E50.h5")

# DeepFakeDetection Test 

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakeDetection/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFake Test 

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFake/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# FaceSwap Test 

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/FaceSwap/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# Face2Face Test 

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/Face2Face/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# NeuralTextures Test 

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/NeuralTextures/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFakewild Test 

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakewild/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DFDC Test 

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DFDC/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# New DeepFake wild Test

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(128, 128, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("best_models/XCE_ALL_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DFP_new/test',# this is the target directory
        target_size=(128, 128),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.predict_generator(test_generator,verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)